In [1]:
import os

In [2]:
%pwd

'/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint'

In [5]:
from finance_complaint.entity.schema import FinanceDataSchema
from pyspark.ml.feature import StandardScaler, VectorAssembler, OneHotEncoder, StringIndexer, Imputer
from pyspark.ml.pipeline import Pipeline
from finance_complaint.config.spark_manager import spark_session
from finance_complaint.exception import FinanceException
from finance_complaint.logger import logging as logger
from finance_complaint.entity.artifact_entity import DataValidationArtifact #, DataTransformationArtifact
# from finance_complaint.entity.config_entity import DataTransformationConfig
from pyspark.sql import DataFrame
# from finance_complaint.ml.feature import FrequencyImputer, DerivedFeatureGenerator
from pyspark.ml.feature import IDF, Tokenizer, HashingTF
from pyspark.sql.functions import col, rand
import os,sys

Reading environment variables
Read Complete!
MongoClient(host=['ac-zciqt62-shard-00-01.uhbxanv.mongodb.net:27017', 'ac-zciqt62-shard-00-02.uhbxanv.mongodb.net:27017', 'ac-zciqt62-shard-00-00.uhbxanv.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='mlcluster', authsource='admin', replicaset='atlas-10vcjq-shard-0', tls=True, server_api=<pymongo.server_api.ServerApi object at 0x11555d6a0>)


25/05/10 17:09:01 WARN Utils: Your hostname, Rahuls-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
25/05/10 17:09:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/10 17:09:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 50951)
Traceback (most recent call last):
  File "/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/venv/lib/python3.13/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/venv/lib/python3.13/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/venv/lib/python3.13/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
    ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rahulshelke/Documents/Data-Science/Data

## config_entity.py

In [6]:
from finance_complaint.entity.config_entity import TrainingPipelineConfig
from dataclasses import dataclass

In [7]:
# Data Transformation Constants
DATA_TRANSFORMATION_DIR = "data_transformation"
DATA_TRANSFORMATION_PIPELINE_DIR = "transformed_pipeline"
DATA_TRANSFORMATION_TRAIN_DIR = "train"
DATA_TRANSFORMATION_FILE_NAME = "finance_complaint"
DATA_TRANSFORMATION_TEST_DIR = "test"
DATA_TRANSFORMATION_TEST_SIZE = 0.3

In [8]:
@dataclass
class DataTransformationConfig:

    def __init__(self, training_pipeline_config: TrainingPipelineConfig) -> None:
        try:
            data_transformation_dir = os.path.join(training_pipeline_config.artifact_dir, DATA_TRANSFORMATION_DIR)
            self.transformed_train_dir = os.path.join(data_transformation_dir, DATA_TRANSFORMATION_TRAIN_DIR)
            self.transformed_test_dir = os.path.join(data_transformation_dir, DATA_TRANSFORMATION_TEST_DIR)
            self.export_pipeline_dir = os.path.join(data_transformation_dir, DATA_TRANSFORMATION_PIPELINE_DIR)
            self.file_name = DATA_TRANSFORMATION_FILE_NAME
            self.test_size = DATA_TRANSFORMATION_TEST_SIZE
        except Exception as e:
            raise FinanceException(e, sys)
        

## artifact_entity.py

In [9]:
@dataclass
class DataTransformationArtifact:
    transformed_train_file_path: str
    exported_pipeline_file_path: str
    transformed_test_file_path: str

## feature.py

In [10]:
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import Param, Params, TypeConverters, HasOutputCols, HasInputCols
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.ml import Estimator
from pyspark.sql import DataFrame
from pyspark.sql.functions import desc
from pyspark.sql.functions import col, abs
from typing import List
from pyspark.sql.types import TimestampType, LongType
from finance_complaint.logger import logging as logger
from finance_complaint.config.spark_manager import spark_session

In [11]:
class FrequencyEncoder(Estimator, HasInputCols, HasOutputCols,
                       DefaultParamsReadable, DefaultParamsWritable):
    frequencyInfo = Param(Params._dummy(), "getfrequencyInfo", "getfrequencyInfo",
                          typeConverter=TypeConverters.toList)
    @keyword_only
    def __init__(self, inputCols: List[str] = None, outputCols:List[str] = None,):
        super(FrequencyEncoder, self).__init__()
        kwargs = self._input_kwargs

        self.frequencyInfo = Param(self, "frequencyInfo", "")
        self._setDefault(frequencyInfo="")
        #self._set(**kwargs)

        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCols: List[str] = None, outputCols: List[str] = None,):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def setInputCols(self, value: List[str]):
        """ 
        Set the value of :py:attr:`inputCol`.
        """
        return self._set(inputCol=value)
    
    def setOutputCols(self, value: List[str]):
        """ 
        Set the value of :py:attr:`outputCol`
        """
        return self._set(outputCols=value)
    
    def setfrequencyInfo(self):
        return self.getOrDefault(self.frequencyInfo)
    
    def _fit(self, dataframe: DataFrame):
        input_columns = self.getInputCols()
        print(f"Input columns: {input_columns}")
        output_columns = self.getOutputCols()
        print(f"Output columns: {output_columns}")
        replace_info = []
        for column, new_column in zip(input_columns, output_columns):
            freq = (dataframe.select(col(column).alias(f'g_{column}')).groupby(f'g_{column}').count().withColumn(new_column, col('count')))
            freq = freq.drop('count')
            logger.info(f"{column} has [{freq.count()}] unique category.")
            replace_info.append(freq.collect())

        self.setfrequencyInfo(frequencyInfo=replace_info)
        estimator = FrequencyEncoderModel(inputCols=input_columns, outputCols=output_columns)
        estimator.setfrequencyInfo(frequencyInfo=replace_info)
        return estimator
    
class FrequencyEncoderModel(FrequencyEncoder, Transformer):

    def __init__(self, inputCols: List[str] = None, outputCols: List[str] = None):
        super(FrequencyEncoderModel, self).__init__(inputCols=inputCols, outputCols=outputCols)

    def _transform(self, dataframe: DataFrame):
        inputCols = self.getInputCols()
        outputCols = self.getOutputCols()

        print(f"Input Columns: {inputCols}")
        print(f"Output Columns: {outputCols}")

        freqInfo = self.getfrequencyInfo()


        for in_col, out_col , freq_info in zip(inputCols, outputCols, freqInfo):
            frequency_dataframe: DataFrame = spark_session.createDataFrame(freq_info)

            columns = frequency_dataframe.columns

            dataframe = dataframe.join(frequency_dataframe,
                                       on= dataframe[in_col] == frequency_dataframe[columns[0]])
            
            dataframe = dataframe.drop(columns[0])

            if out_col not in dataframe.columns:
                dataframe = dataframe.withColumn(out_col, col(columns[1]))
                dataframe= dataframe.drop(columns[1])

        return dataframe
    
class DerivedFeatureGenerator(Transformer, HasInputCols, HasOutputCols,
                              DefaultParamsReadable, DefaultParamsWritable):
    @keyword_only
    def __init__(self, inputCols: List[str] = None, outputCols: List[str] = None,):
        super(DerivedFeatureGenerator, self).__init__()
        kwargs = self._input_kwargs
        # self._set(**kwargs)
        self.second_within_day = 60 * 60 * 24
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCols: List[str] = None, outputCols: List[str] = None,):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def setInputCols(self, value: List[str]):
        """ 
        Sets the value of :py:attr:`inputCol`.
        """
        return self._set(inputCols=value)
    
    def setOutputCols(self, value: List[str]):
        """ 
        Sets the value of :py:attr:`outputCol`.
        """
        return self._set(outputCols=value)
    
    def _fit(self, dataframe: DataFrame):
        return self

    def _transform(self, dataframe: DataFrame):
        inputCols = self.getInputCols()

        for column in inputCols:
            dataframe = dataframe.withColumn(column, col(column).cast(TimestampType()))

        dataframe = dataframe.withColumn(self.getOutputCols()[0],
                                         abs(col(inputCols[1]).cast(LongType()) - col(inputCols[0]).cast(LongType()))/ (self.second_within_day))
        return dataframe
    
class FrequencyImputer(
    Estimator,
    HasInputCols,
    HasOutputCols,
    DefaultParamsReadable,
    DefaultParamsWritable
):
    topCategories = Param(Params._dummy(), "getTopCategories", "getTopCategories", 
                          typeConverter=TypeConverters.toListString)
    
    @keyword_only
    def __init__(self, inputCols: List[str] = None, outputCols: List[str] = None):
        super(FrequencyImputer, self).__init__()
        self.topCategories = Param(self, "topCategories", "")
        self._setDefault(topCategories="")
        kwargs = self._input_kwargs
        print(kwargs)
        self.setParams(**kwargs)


    @keyword_only
    def setParams(self, inputCols: List[str] = None, outputCols: List[str] = None, ):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def setTopCategories(self, value: List[str]):
        return self._set(topCategories=value)

    def getTopCategories(self):
        return self.getOrDefault(self.topCategories)
    
    def setInputCols(self, value: List[str]):
        """ 
        set the value of :py:attr:`inputCol`
        """
        return self._set(inputCols=value)
    
    def setOutputCols(self, value: List[str]):
        """ 
        set the value of :py:attr:`outputCol`
        """
        return self._set(outputCols=value)
    
    def _fit(self, dataset: DataFrame):
        inputCols = self.getInputCols()
        topCategories = []
        for column in inputCols:
            categoryCountByDesc = dataset.groupBy(column).count().filter(f'{column} IS NOT NULL').sort(desc('count'))
            topCat = categoryCountByDesc.take(1)[0][column]
            topCategories.append(topCat)
        
        self.setTopCategories(value=topCategories)

        estimator = FrequencyImputerModel(inputCols = self.getInputCols(),
                                          outputCols = self.getOutputCols())
        
        estimator.setTopCategories(value=topCategories)
        return estimator
    
class FrequencyImputerModel(FrequencyImputer, Transformer):

    def __init__(self, inputCols: List[str] = None, outputCols: List[str] = None,):
        super(FrequencyImputerModel, self).__init__(inputCols=inputCols, outputCols=outputCols)

    def _transform(self, dataset: DataFrame):
        topCategories = self.getTopCategories()
        outputCols = self.getOutputCols()

        updateMissingValues = dict(zip(outputCols, topCategories))

        inputCols = self.getInputCols()

        for outputColumn, inputColumn in zip(outputCols, inputCols):
            dataset = dataset.withColumn(outputColumn, col(inputColumn))
            # print(dataset.columns)
            # print(outputColumn, inputColumn)

        dataset = dataset.na.fill(updateMissingValues)

        return dataset

## data transformation step by step

In [12]:
data_validation_artifact = DataValidationArtifact
data_transformation_config = DataTransformationConfig
schema=FinanceDataSchema()

In [13]:
# def read_data() -> DataFrame:
#     try:
#         file_path = DataValidationArtifact().accepted_file_path
#         dataframe : DataFrame = spark_session.read.parquet(file_path)
#         dataframe.printSchema()
#         return dataframe
#     except Exception as e:
#         raise FinanceException(e, sys)

In [14]:
stages = []
derived_features = DerivedFeatureGenerator(
    inputCols=schema.derived_input_features,
    outputCols=schema.derived_output_features
)

stages.append(derived_features)

In [15]:
stages

[DerivedFeatureGenerator_2a8585211179]

In [16]:
imputer = Imputer(inputCol=schema.numerical_columns[-1], outputCol=schema.im_numerical_columns[-1])

stages.append(imputer)

In [17]:
frequency_imputer = FrequencyImputer(inputCols=schema.one_hot_encoding_features, outputCols=schema.im_one_hot_encoding_features)

stages.append(frequency_imputer)

{'inputCols': ['company_response', 'consumer_consent_provided', 'submitted_via'], 'outputCols': ['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via']}


In [18]:
string_indexer = StringIndexer(inputCols=schema.im_one_hot_encoding_features, outputCols=schema.string_indexer_one_hot_features)

stages.append(string_indexer)

In [19]:
one_hot_encoder = OneHotEncoder(inputCols=schema.string_indexer_one_hot_features,
                                outputCols=schema.tf_one_hot_encoding_features)

stages.append(one_hot_encoder)

In [20]:
tokenizer = Tokenizer(inputCol=schema.tfidf_features[0], outputCol="words")

stages.append(tokenizer)

In [21]:
hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures", numFeatures=40)

stages.append(hashing_tf)

In [22]:
idf = IDF(inputCol= hashing_tf.getOutputCol(), outputCol=schema.tf_tfidf_features[0])

stages.append(idf)

In [23]:
vector_assembler = VectorAssembler(inputCols=schema.input_features,
                                    outputCol=schema.vector_assembler_output)

stages.append(vector_assembler)

In [24]:
standard_scaler = StandardScaler(inputCol=schema.vector_assembler_output,
                                    outputCol=schema.scaled_vector_input_features)
stages.append(standard_scaler)

In [25]:
pipeline = Pipeline(
    stages=stages
)

In [26]:
pipeline

Pipeline_4c607d74047c

In [27]:
# dataframe: DataFrame = read_data()

In [28]:

# logger.info(f"Number of row: [{dataframe.count()}] and column: [{len(dataframe.columns)}]")

# test_size = self.data_tf_config.test_size

# logger.info(f"Splitting dataset into train and test set using ration : {1 - test_size}:{test_size}")

# train_dataframe, test_dataframe = dataframe.randomSplit([1 - test_size, test_size])

# logger.info(f"Train dataset has number of row: [{train_dataframe.count()}] and"
#             f"column: [{len(train_dataframe.columns)}]")

# logger.info(f"Test dataset has number of row: [{test_dataframe.count()}] and"
#             f"column: [{len(test_dataframe.columns)}]")

# pipline = self.get_data_transformation_pipeline()

# print(f"train_dataframe: {train_dataframe}")

# transformed_pipeline = pipline.fit(train_dataframe)

# required_column = [self.schema.scaled_vector_input_features, self.schema.target_column]

# transformed_train_dataframe = transformed_pipeline.transform(train_dataframe)
# transformed_train_dataframe = transformed_train_dataframe.select(required_column)

# transformed_test_dataframe = transformed_pipeline.transform(test_dataframe)
# transformed_test_dataframe = transformed_test_dataframe.select(required_column)

## data_transformation.py

In [29]:
class DataTransformation:

    def __init__(self, data_validation_artifact: DataValidationArtifact,
                 data_transformation_config: DataTransformationConfig,
                 schema=FinanceDataSchema()
                 ):
        try:
            self.data_val_artifact = data_validation_artifact
            self.data_tf_config = data_transformation_config
            self.schema = schema
        except Exception as e:
            raise FinanceException(e, sys)
        
    def read_data(self) -> DataFrame:
        try:
            file_path = self.data_val_artifact.accepted_file_path

            print(f"-- file_path: {file_path}")
            
            dataframe : DataFrame = spark_session.read.parquet(file_path)
            dataframe.printSchema()
            return dataframe
        except Exception as e:
            raise FinanceException(e, sys)
        
    def get_data_transformation_pipeline(self,) -> Pipeline:
        try:
            stages = []
            derived_features = DerivedFeatureGenerator(
                inputCols=self.schema.derived_input_features,
                outputCols=self.schema.derived_output_features
            )
            stages.append(derived_features)

            imputer = Imputer(inputCol=self.schema.numerical_columns[-1], outputCol=self.schema.im_numerical_columns[-1])

            stages.append(imputer)

            frequency_imputer = FrequencyImputer(inputCols=self.schema.one_hot_encoding_features, outputCols=self.schema.im_one_hot_encoding_features)
            stages.append(frequency_imputer)

            # for im_one_hot_feature, string_indexer_col in zip(self.schema.im_one_hot_encoding_features, self.schema.string_indexer_one_hot_features):

            string_indexer = StringIndexer(inputCols=self.schema.im_one_hot_encoding_features, outputCols=self.schema.string_indexer_one_hot_features) 
            stages.append(string_indexer)

            one_hot_encoder = OneHotEncoder(inputCols=self.schema.string_indexer_one_hot_features,
                                            outputCols=self.schema.tf_one_hot_encoding_features)

            stages.append(one_hot_encoder)

            tokenizer = Tokenizer(inputCol=self.schema.tfidf_features[0], outputCol="words")

            stages.append(tokenizer)

            hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures", numFeatures=40)

            stages.append(hashing_tf)

            idf = IDF(inputCol= hashing_tf.getOutputCol(), outputCol=self.schema.tf_tfidf_features[0])

            stages.append(idf)

            vector_assembler = VectorAssembler(inputCols=self.schema.input_features,
                                               outputCol=self.schema.vector_assembler_output)
            
            stages.append(vector_assembler)

            standard_scaler = StandardScaler(inputCol=self.schema.vector_assembler_output,
                                             outputCol=self.schema.scaled_vector_input_features)
            stages.append(standard_scaler)

            pipeline = Pipeline(
                stages=stages
            )

            logger.info(f"Data transformation pipeline: [{pipeline}]")
            print(pipeline.stages)
            return pipeline
    
        except Exception as e:
            raise FinanceException(e, sys)
        

    def initiate_data_transformation(self) -> DataTransformationArtifact:

        try:
            logger.info(f">>>>>>>>>>>>>>>Stated data transformation<<<<<<<<<<<<<<<<")
            dataframe: DataFrame = self.read_data()
            logger.info(f"Number of row: [{dataframe.count()}] and column: [{len(dataframe.columns)}]")

            test_size = self.data_tf_config.test_size

            logger.info(f"Splitting dataset into train and test set using ration : {1 - test_size}:{test_size}")

            train_dataframe, test_dataframe = dataframe.randomSplit([1 - test_size, test_size])

            logger.info(f"Train dataset has number of row: [{train_dataframe.count()}] and"
                        f"column: [{len(train_dataframe.columns)}]")
            
            logger.info(f"Test dataset has number of row: [{test_dataframe.count()}] and"
                        f"column: [{len(test_dataframe.columns)}]")
            
            pipline = self.get_data_transformation_pipeline()

            print(f"train_dataframe: {train_dataframe}")

            transformed_pipeline = pipline.fit(train_dataframe)

            required_column = [self.schema.scaled_vector_input_features, self.schema.target_column]

            transformed_train_dataframe = transformed_pipeline.transform(train_dataframe)
            transformed_train_dataframe = transformed_train_dataframe.select(required_column)

            transformed_test_dataframe = transformed_pipeline.transform(test_dataframe)
            transformed_test_dataframe = transformed_test_dataframe.select(required_column)

            export_pipeline_file_path = self.data_tf_config.export_pipeline_dir
            os.makedirs(export_pipeline_file_path, exist_ok=True)
            os.makedirs(self.data_tf_config.transformed_test_dir, exist_ok=True)
            os.makedirs(self.data_tf_config.transformed_train_dir, exist_ok=True)

            transformed_train_data_file_path = os.path.join(self.data_tf_config.transformed_train_dir,
                                                            self.data_tf_config.file_name)
            
            transformed_test_data_file_path = os.path.join(self.data_tf_config.transformed_test_dir,
                                                            self.data_tf_config.file_name)
            logger.info(f"Saving transformation pipeline at: [{export_pipeline_file_path}]")

            transformed_pipeline.save(export_pipeline_file_path)
            logger.info(f"Saving tranformation train data at: [{transformed_train_data_file_path}]")
            print(transformed_train_dataframe.count(), len(transformed_train_dataframe.columns))
            transformed_train_dataframe.write.parquet(transformed_train_data_file_path)

            logger.info(f"Saving transformation test data at: [{transformed_test_data_file_path}]")
            print(transformed_test_dataframe.count(), len(transformed_train_dataframe.columns))
            transformed_test_dataframe.write.parquet(transformed_test_data_file_path)

            data_tf_artifact = DataTransformationArtifact(
                transformed_train_file_path = transformed_train_data_file_path,
                transformed_test_file_path = transformed_test_data_file_path,
                exported_pipeline_file_path = export_pipeline_file_path,
            )
            logger.info(f"Data tranformation artifact: [{data_tf_artifact}]")

            return data_tf_artifact
        
        except Exception as e:
            raise FinanceException(e, sys)

## training.py

In [30]:
from finance_complaint.exception import FinanceException
from finance_complaint.logger import logging as logger
from finance_complaint.entity.schema import FinanceDataSchema

from finance_complaint.entity.config_entity import (DataIngestionConfig, 
                                                    DataValidationConfig,
                                                    # DataTransformationConfig
                                                    )
from finance_complaint.entity.artifact_entity import (DataIngestionArtifact,
                                                      DataValidationArtifact,
                                                    #   DataTransformationArtifact
                                                      )

from finance_complaint.components.data_ingestion import DataIngestion
from finance_complaint.components.data_validation import DataValidation
# from finance_complaint.components.data_transformation import DataTransformation

import sys, os


In [31]:
class TrainingPipeline:
    """ 
    this is a training pipeline comprising each components from flow chart
    """
    def __init__(self, training_pipeline_config: TrainingPipelineConfig):
        self.training_pipeline_config: TrainingPipelineConfig = training_pipeline_config

    def start_data_ingestion(self) -> DataIngestionArtifact:
        """
        responsible for starting data ingestion 
        """
        try:
            data_ingestion_config = DataIngestionConfig(training_pipeline_config=self.training_pipeline_config)
            data_ingestion = DataIngestion(data_ingestion_config=data_ingestion_config)
            data_ingestion_artifact = data_ingestion.initiate_data_ingestion()
            return data_ingestion_artifact
        
        except Exception as e:
            logger.debug(f"Error: {e}")
            raise FinanceException(e, sys)
        
    def start_data_validation(self, data_ingestion_artifact: DataIngestionArtifact) -> DataValidationArtifact:
        """
        takes ingested data artifact and validate that data by generating validation artifact
        """
        try:
            data_validation_config = DataValidationConfig(training_pipeline_config=self.training_pipeline_config)
            data_validation = DataValidation(
                data_validation_config=data_validation_config,
                data_ingestion_artifact=data_ingestion_artifact
            )
            data_validation_artifact = data_validation.initiate_data_validation()
            return data_validation_artifact
        except Exception as e:
            raise FinanceException(e, sys)
        
    def start_data_transformation(self, data_validation_artifact: DataValidationArtifact) -> DataTransformationArtifact:
        """ 
        responsible for data transformation 
        """
        try:
            data_transformation_config = DataTransformationConfig(training_pipeline_config=self.training_pipeline_config)
            data_transformation = DataTransformation(data_validation_artifact=data_validation_artifact,
                                                     data_transformation_config=data_transformation_config)
            
            data_transformation_artifacts = data_transformation.initiate_data_transformation()
            return data_transformation_artifacts
        except Exception as e:
            raise FinanceException(e, sys)
        
    def start(self):
        try:
            # initalizating data ingestion
            data_ingestion_artifact = self.start_data_ingestion()
            # initalizing data validation
            data_validation_artifact = self.start_data_validation(data_ingestion_artifact=data_ingestion_artifact)
            # initalizing data transformation
            data_transformation_artifact = self.start_data_transformation(data_validation_artifact=data_validation_artifact)

        except Exception as e:
            raise FinanceException(e, sys)    


## train.py

In [32]:
# from finance_complaint.pipeline.training import TrainingPipeline
# from finance_complaint.entity.config_entity import TrainingPipelineConfig

if __name__ == "__main__":
    training_pipeline_config = TrainingPipelineConfig()
    training_pipeline = TrainingPipeline(training_pipeline_config=training_pipeline_config)
    training_pipeline.start()

Row: [10000] Column: [12]
Expected Column: ['consumer_disputed', 'company_response', 'consumer_consent_provided', 'submitted_via', 'issue', 'date_sent_to_company', 'date_received']
Present Columns: ['company', 'company_response', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'submitted_via', 'timely', 'zip_code']
-- file_path: artifact/20250510_170857/data_validation/accepted_data/finance_complaint
root
 |-- company: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- date_received: string (nullable = true)
 |-- date_sent_to_company: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- product: string (nullable = true)
 |-- state: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- timely: string (nullable = true)
 |-- zip_code: string (nullable =

7055 2


2945 2


25/05/10 19:27:11 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 964289 ms exceeds timeout 120000 ms
25/05/10 19:27:11 WARN SparkContext: Killing executors is not supported by current scheduler.
25/05/10 19:31:28 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$